In [6]:
import os
import polars as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import gc

from tqdm import tqdm
from prj.config import DATA_DIR


BASE_PATH = DATA_DIR

limit=10

for partition in tqdm(range(0, 10)):
    part_path = BASE_PATH / 'train_ffill.parquet' / f'partition_id={partition}'
    os.makedirs(part_path, exist_ok=True)
    if partition > 0:
        old = pl.read_parquet(BASE_PATH / 'train.parquet' / f'partition_id={partition-1}' / 'part-0.parquet').with_columns(pl.lit(partition-1).alias('part')).sort('date_id', 'time_id')
    else:
        old = pl.DataFrame({})
        
    curr = pl.read_parquet(BASE_PATH / 'train.parquet' / f'partition_id={partition}' / 'part-0.parquet').with_columns(pl.lit(partition).alias('part')).sort('date_id', 'time_id')
    features = [col for col in curr.columns if col not in ['date_id', 'time_id', 'symbol_id', 'part']]

    old.vstack(curr).with_columns(
        pl.col(features).fill_nan(None).fill_null(strategy='forward', limit=limit).over('symbol_id').name.keep()
    ).filter(
        pl.col('part') == partition
    ).drop('part')\
    .write_parquet(part_path / 'part-0.parquet')
    
    del old, curr
    gc.collect()

100%|██████████| 10/10 [08:40<00:00, 52.00s/it]
